# 4-3 AutoGraph的使用规范

有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

* 动态计算图易于调试，编码效率较高，但执行效率偏低。

* 静态计算图执行效率很高，但较难调试。

* **而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。**

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。

我们将着重介绍Autograph的编码规范和Autograph转换成静态图的原理。

并介绍使用tf.Module来更好地构建Autograph。

本篇我们介绍使用Autograph的编码规范。

### 一、Autograph编码规范总结


* 1、被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

* 2、避免在@tf.function修饰的函数内部定义tf.Variable. 

* 3、被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

### 二、Autograph编码规范解析

 **1、被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。**

In [2]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    # np.random.randn函数返回一个或一组样本，具有标准正态分布
    a = np.random.randn(3, 3)
    tf.print(a)
    
@tf.function
def tf_random():
    # tf.random.normal 服从指定正态分布的序列
    a = tf.random.normal((3, 3))
    tf.print(a)

In [3]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.37556616,  1.06691645,  1.70576025],
       [ 0.86436862,  0.43722011, -1.52256556],
       [ 0.46070659,  2.10384045,  1.06085463]])
array([[-0.37556616,  1.06691645,  1.70576025],
       [ 0.86436862,  0.43722011, -1.52256556],
       [ 0.46070659,  2.10384045,  1.06085463]])


In [4]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[-0.502602816 -0.489105225 -0.562851548]
 [0.39553681 0.248162314 -0.186708838]
 [0.0281172413 1.21298826 2.1669879]]
[[-1.40998316 0.863256097 -0.448851466]
 [-0.0775592774 0.504335105 -2.06567192]
 [-1.05170166 -0.412114412 0.273190349]]


**2、避免在@tf.function修饰的函数内部定义tf.Variable.**

In [9]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return x

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [12]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错
#inner_var()
inner_var()

ValueError: in converted code:

    <ipython-input-10-60fa9b0fb738>:3 inner_var  *
        x = tf.Variable(1.0,dtype = tf.float32)
    /Users/alan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /Users/alan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /Users/alan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /Users/alan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


**3、被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。**

In [14]:
tensor_list = []

@tf.function # 加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]


In [15]:
tensor_list = []

# @tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]
